In [32]:
import pandas as pd
import numpy as np
import seaborn as sns
import datetime as datetime
import os
from constants import *
from trip_file import TripFile
from trip import Trip


### Read all files

In [3]:
taarif = pd.read_csv("../files/taarif.csv")
taarif = taarif.drop(0).reset_index().drop(columns=["index"]) # first row is nulls
new_drivers = pd.read_csv("../files/new_drivers.csv", index_col=0)
drivers_with_kviut = pd.read_csv("../files/drivers_with_kviut.csv",index_col=0)

In [4]:


def preprocess_drivers(drivers: pd.DataFrame, copy=False) -> pd.DataFrame:
    df = drivers
    if copy:
        df = drivers.copy()

    gender_mapping = {"F": FEMALE,
                    "M":MALE,
                    "m":MALE,
                    "male":MALE,
                    "boy":MALE,
                    "unknown":MALE,
                    'woman':FEMALE,
                    'girl':FEMALE,
                    'none':MALE,
                    'female':FEMALE}

    # Make preprocess
    df.gender.fillna(MALE, inplace=True)
    df.gender = df.gender.apply(lambda x: gender_mapping[x])
    df.birthdate = pd.to_datetime(df.birthdate, format="mixed")
    drivers.birthdate.fillna(datetime.datetime.now(),inplace=True)

    return df

def preprocess_taarif(taarif_df: pd.DataFrame, copy=False) -> pd.DataFrame:
        df = taarif_df
        if copy:
            df = new_drivers.copy()

        return df


# Concat the two tables of drivers with and without kviut
def concat_drivers_tables(new_drivers, drivers_with_kviut):
    drivers_with_kviut["kviut"] = 1
    drivers = pd.concat([new_drivers, drivers_with_kviut])
    drivers["kviut"].fillna(0, inplace=True)
    return drivers
    

In [5]:
drivers = concat_drivers_tables(new_drivers, drivers_with_kviut)
drivers = preprocess_drivers(drivers, copy=True)

In [6]:
from tqdm import tqdm
def get_trip_files(folder, limit=100):
    files = np.array(list(os.listdir(folder)))
    if limit:
        files = files[:limit]
    files = np.array([name if not name[-5] == ")" else name[:-7]+".csv" for name in files])
    print("Processing files....")
    return np.array([TripFile(os.path.join(folder,file)) for file in tqdm(files)])

In [7]:

def calculate_trip_salary(taarif, drivers, customer, trip_km, trip_start, trip_end):
    info = ""
    
    basic_rate = taarif[taarif.customer == "blalbalbla"]["basic_taarif"]
    if basic_rate.empty:
        info = "Company not found"
        return -1
    
    if not trip_start or not trip_end:
        info = "possible weekend bonus\n"
    


In [31]:
def check_if_weekend(start_time, end_time):
    return False

def check_if_night(start_time, end_time):
    return False

def calculate_trips_cost(trips_with_fares):
    
    df = trips_with_fares.copy()
    is_weekend = df.apply(lambda x: check_if_weekend(x.start_time, x.end_time), axis=1)
    is_night = df.apply(lambda x: check_if_night(x.start_time, x.end_time), axis=1)
    basic_pay = df.km * df.basic_taarif
    extra_pay = max(0, df.km - 200) * df.extra_milage
    payment = basic_pay + extra_pay

    bonus_precentage = is_weekend.apply(int) * df.weekend_bonus + is_night.apply(int) * df.night_bonus
    payment = payment.apply(lambda x: )


def create_main_table():
    # rates = Rates("files/taarif.csv")
    table = pd.DataFrame(columns=["id","month","year","total_income","total_km","gender","age","vetek","kviut","info"])
    trip_files = get_trip_files("../files/trips_data")
    for trip_file in trip_files:
        
        trip_df = trip_file.get_data_frame()
        # print(trip_df)
        trips_with_fares = pd.merge(trip_df, taarif, on=["customer"],how="left")

        
        break
    january = "january"


create_main_table()

    


Processing files....


  0%|          | 0/100 [00:00<?, ?it/s]

100%|██████████| 100/100 [00:00<00:00, 207.39it/s]

0     13446.254321
1              NaN
2       725.823979
3              NaN
4       771.501038
5      2759.827000
6      3870.011305
7              NaN
8              NaN
9      2649.688860
10             NaN
11       81.706881
12     3012.519807
13             NaN
14             NaN
15     1576.672074
16      419.454279
17     1083.657110
18     3671.935676
19     9490.446214
20             NaN
21      634.097917
22     1892.007950
23     5971.314435
24     2701.948601
25     3040.771004
26             NaN
27     1362.857100
28             NaN
29     2153.632194
30     1942.666302
31    11034.528199
32             NaN
33             NaN
34     4673.267702
35             NaN
36    12001.648431
37     4275.969490
38             NaN
39             NaN
40      512.527499
41             NaN
42             NaN
43             NaN
44      223.529437
45             NaN
46             NaN
47    13268.482525
48     1720.609531
49             NaN
50     1368.856742
51     4460.832775
52          

In [ ]:
df1 = pd.DataFrame({"A":[0,1,2,3],
                    "B":[1,2,3,4]})

df1.A + df1.B